In [21]:
import pandas as pd

pd.set_option("display.max_rows", 300)
import rapidfuzz
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import awswrangler as wr
import boto3
import copy
import itertools
from pprint import pprint


In [22]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"
srcBucketName = "multilabel_df"
labelDf = wr.s3.read_parquet(
    path=f"s3://{srcBucketName}/",
    dataset=True
)
labelDf

,vision_bonnet,vision_bumper_front,vision_engine,vision_grille,vision_headlamp_lh,vision_headlamp_rh,vision_door_front_lh,vision_door_front_rh,vision_bumper_rear,vision_misc,...,Assembly_Type,Vehicle_Still_Driveable,NCB_Stat,Claim_Type,Vehicle_Type,Sum_Insured,Repairer,Repairer_Apprv_Count,Collision_With,Handling_Insurer
0,1,1,1,1,0,0,0,0,0,0,...,3,0,1,OD,SUV - 5 Dr,63000.0,Million Car Auto Specialist Sdn Bhd (HQ),1163,Private Vehicle,AXA Affin General Insurance Berhad
1,1,1,1,0,1,1,1,1,1,1,...,3,0,1,OD,Saloon - 4 Dr,39000.0,Top Diversified Sdn Bhd (Top Diversified),1058,Private Vehicle,Allianz General Insurance Company (Malaysia) B...
2,1,1,0,0,1,1,0,0,0,0,...,3,0,1,OD,Hatchback - 5 Dr,8000.0,The Apple Auto Services & Tyres Sdn Bhd (The A...,6031,Private Vehicle,Etiqa General Insurance Berhad
3,1,1,1,1,0,0,0,1,0,1,...,3,0,1,OD,Pickup - 4 Dr Dbl.Cab,16000.0,Bengkel Baru Seng Soon Sdn Bhd (HQ),1686,Others,MSIG INSURANCE (MALAYSIA) BHD
4,1,1,1,1,0,0,0,0,0,0,...,3,0,1,OD,Saloon - 4 Dr,44000.0,LB Performance Auto Works Sdn Bhd (HQ),600,Private Vehicle,Zurich General Insurance Malaysia Berhad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595704,0,0,0,0,0,0,0,0,0,0,...,3,1,0,OD KFK,Saloon - 4 Dr,48000.0,New Way Motor Workshop Sdn Bhd (HQ),3234,Private Vehicle,Tokio Marine Insurans (Malaysia) Berhad
595705,0,0,0,0,0,0,0,0,0,0,...,3,1,0,OD EXW,Saloon - 4 Dr,40000.0,Tan Chong Ekspres Auto Servis Sdn Bhd (Kuantan...,0,,RHB Insurance Berhad
595706,0,0,0,0,0,0,0,0,0,0,...,2,1,0,OD EXW,Saloon - 4 Dr,40000.0,Tan Chong Ekspres Auto Servis Sdn Bhd (K.Teren...,0,,RHB Insurance Berhad
595707,0,0,0,0,0,0,0,0,0,0,...,3,1,0,OD EXW,Saloon - 4 Dr,40000.0,Tan Chong Ekspres Auto Servis Sdn Bhd (ALOR SE...,0,,RHB Insurance Berhad


In [23]:
mappingDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/tmp/complete_view_mapping.csv")

In [24]:
mappingDf["lvl_3_desc"] = mappingDf["lvl_3_desc"].str.strip()

In [25]:
view = ["front_view", "rear_view", "front_view_left", "front_view_right", "rear_view_left", "rear_view_right"]
viewToPart = dict()
for v in view:
    partsInView = mappingDf[mappingDf[v] == 1]["lvl_3_desc"].unique().tolist()
    viewToPart[v] = ["vision_" + x.replace(" ", "_") for x in partsInView]
pprint(viewToPart)
    

{'front_view': ['vision_bonnet',
                'vision_bumper_front',
                'vision_engine',
                'vision_front_panel',
                'vision_grille',
                'vision_headlamp_lh',
                'vision_headlamp_rh',
                'vision_misc',
                'vision_non_external',
                'vision_windscreen_front'],
 'front_view_left': ['vision_bonnet',
                     'vision_bumper_front',
                     'vision_door_front_lh',
                     'vision_door_rear_lh',
                     'vision_engine',
                     'vision_fender_front_lh',
                     'vision_front_panel',
                     'vision_grille',
                     'vision_headlamp_lh',
                     'vision_non_external',
                     'vision_misc',
                     'vision_wheel',
                     'vision_windscreen_front'],
 'front_view_right': ['vision_bonnet',
                      'vision_bumper_front',
    

In [26]:
vehicleType= "Saloon - 4 Dr"
localFileMetadata = f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/imgs_metadata/{vehicleType}.parquet"

In [27]:
localFilesDf = pd.read_parquet(localFileMetadata)
localFilesDf["StdDocDesc"] = localFilesDf["StdDocDesc"].str.replace(" ", "_").str.lower()
localFilesDf.sort_values(by="CaseID", inplace=True)
localFilesDf

,CaseID,iDOCID,StdDocDesc,filename,url
1581,10000038,438742010.0,front_view,10000038_438742010.JPG,http://192.168.1.4:8888/buckets/raw_imgs/10000...
2285288,10000038,438742000.0,rear_view_right,10000038_438742000.JPG,http://192.168.1.4:8888/buckets/raw_imgs/10000...
280588,10000038,438742006.0,front_view_left,10000038_438742006.JPG,http://192.168.1.4:8888/buckets/raw_imgs/10000...
1474120,10000038,438742004.0,rear_view,10000038_438742004.JPG,http://192.168.1.4:8888/buckets/raw_imgs/10000...
878715,10000038,438742078.0,front_view_right,10000038_438742078.JPG,http://192.168.1.4:8888/buckets/raw_imgs/10000...
...,...,...,...,...,...
789736,13076951,595070513.0,front_view_left,13076951_595070513.JPG,http://192.168.1.4:8888/buckets/raw_imgs/13076...
1387864,13076951,595070498.0,front_view_right,13076951_595070498.JPG,http://192.168.1.4:8888/buckets/raw_imgs/13076...
2196298,13076951,595070634.0,rear_view_left,13076951_595070634.JPG,http://192.168.1.4:8888/buckets/raw_imgs/13076...
2794423,13076951,595070636.0,rear_view_right,13076951_595070636.JPG,http://192.168.1.4:8888/buckets/raw_imgs/13076...


In [28]:
viewDfMap = dict()
for v, parts in viewToPart.items():
    tempDf = labelDf[parts + ["CaseID"]]
    tempDf["view"] = v
    viewFilesDf = localFilesDf[localFilesDf["StdDocDesc"] == v]
    viewFilesDf.drop_duplicates(subset=["CaseID"], inplace=True)
    tempDf = tempDf.merge(viewFilesDf[["filename", "CaseID"]], on="CaseID")
    viewDfMap[v] = tempDf


/tmp/ipykernel_64069/903975900.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDf["view"] = v
/tmp/ipykernel_64069/903975900.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  viewFilesDf.drop_duplicates(subset=["CaseID"], inplace=True)
/tmp/ipykernel_64069/903975900.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [29]:
for view, df in viewDfMap.items():
    imgLabelFilename = f"{vehicleType}_{view}_img_labels.csv"
    wr.s3.to_csv(
        df=df,
    path=f"s3://imgs_labels_2/{imgLabelFilename}",
    

    )

In [30]:
for view, df in viewDfMap.items():
    imgLabelFilename = f"{vehicleType}_{view}_img_labels.csv"
    df = wr.s3.read_csv(
    path=f"s3://imgs_labels_2/{imgLabelFilename}",

    )
    print(df)

      Unnamed: 0  vision_bonnet  vision_bumper_front  vision_engine  \
0              0              1                    1              0   
1              1              1                    1              1   
2              2              1                    1              1   
3              3              1                    1              1   
4              4              1                    1              1   
...          ...            ...                  ...            ...   
4983        4983              0                    0              0   
4984        4984              0                    0              0   
4985        4985              0                    0              0   
4986        4986              0                    0              0   
4987        4987              0                    0              0   

      vision_front_panel  vision_grille  vision_headlamp_lh  \
0                      0              1                   0   
1                    